In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from tqdm.notebook import tqdm
from pretty_midi import PrettyMIDI

from src.db import reference_sets as db
from src.utils import strings
from src.evaluation import mgeval

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# prepare database
db.create_tables()
set_id = db.store_ref_set('theorytab', 17752, 4, 'https://github.com/wayne391/symbolic-musical-datasets, melodies crawled from https://www.hooktheory.com/theorytab')

In [3]:
# batch evaluation of all reference midis + store results in db (only similarity distances or also absolute values?)

set_name = 'theorytab' 

source_folder = '../data/reference_data/' + set_name
i = 0
for root, dirs, files in tqdm(os.walk(source_folder)):
    for file in files:
        if "call" in file and "log" not in file: 
            pair_number = file[0:2]
            call_file = root + "/" + file
            response_file = root + "/" + pair_number + "_response.mid"
            pm = PrettyMIDI(call_file)
            call_analysis = mgeval.analyze_midi(call_file, 4)
            response_analysis = mgeval.analyze_midi(response_file, 4)
            similarity_distances = mgeval.calc_distances(call_analysis, response_analysis)

            song_name = strings.remove_prefix(root, source_folder + "\\")
            print(song_name)
            db.store_ref_data(set_id, song_name, pair_number, **similarity_distances)

0it [00:00, ?it/s]

a_a-day-to-remember_downfall-of-us-all_intro-and-verse
0.24974747474747475 0.33225108225108224 0.0825036075036075
a_a-day-to-remember_downfall-of-us-all_pre-chorus-and-chorus
0.4471590909090909 0.4260489510489511 0.02111013986013982
a_a-day-to-remember_downfall-of-us-all_pre-chorus-and-chorus
0.34069264069264066 0.376555023923445 0.03586238323080432
a_a-g-cook_superstar_chorus
0.33695652173913043 0.31 0.026956521739130435
a_a-g-cook_superstar_pre-chorus-and-chorus
0.2691077441077441 0.28127272727272723 0.012164983164983156
a_a-ha_take-on-me_chorus
1.0713636363636363 1.0713636363636363 0.0
C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
a_a-ha_take-on-me_chorus
0.8928030303030302 0.0 0.8928030303030302
a_a-ha_take-on-m

KeyboardInterrupt: 

In [ ]:
# calculate the average or q1, median, q3 + store it as reference for normalization
# => might be worth changing architecture to object orientation here (creating an instance of the evaluation class, where you can set certain reference parameters)